In [3]:
#2021.06.24. THUR
#Hankyeong

#00. 패키지 호출
import numpy as np 
import pandas as pd
import re 
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

#00-1. warning message ignore
warnings.filterwarnings(action='ignore')

#11. IMDB 영화평 감성분석하기. (데이터셋 탐색 및 전처리)
#(1) 데이터셋 불러오기. 
review_train = pd.read_csv('../../data/word2vec_nlp_tutorial_labeledtraindata.tsv', sep='\t')
review_test = pd.read_csv("../../data/word2vec_nlp_tutorial_testData.tsv", sep='\t')

#(2) train 데이터셋 파악하기. 
#①
review_train

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...
24996,5064_1,0,I don't believe they made this film. Completel...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...


In [4]:
#②
review_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [5]:
#③
review_train['review'][0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [6]:
#(3) test 데이터셋 파악하기.
#①
review_test

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...
...,...,...
24995,2155_10,"Sony Pictures Classics, I'm looking at you! So..."
24996,59_10,I always felt that Ms. Merkerson had never got...
24997,2531_1,I was so disappointed in this movie. I am very...
24998,7772_8,"From the opening sequence, filled with black a..."


In [7]:
#②
review_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      25000 non-null  object
 1   review  25000 non-null  object
dtypes: object(2)
memory usage: 390.8+ KB


In [8]:
#③
review_test['review'][0]

"Naturally in a film who's main themes are of mortality, nostalgia, and loss of innocence it is perhaps not surprising that it is rated more highly by older viewers than younger ones. However there is a craftsmanship and completeness to the film which anyone can enjoy. The pace is steady and constant, the characters full and engaging, the relationships and interactions natural showing that you do not need floods of tears to show emotion, screams to show fear, shouting to show dispute or violence to show anger. Naturally Joyce's short story lends the film a ready made structure as perfect as a polished diamond, but the small changes Huston makes such as the inclusion of the poem fit in neatly. It is truly a masterpiece of tact, subtlety and overwhelming beauty."

In [9]:
#(4) train 데이터셋의 review 변수 전처리하기. 
review_train['review'] = review_train['review'].apply(lambda x: re.sub('[^A-Za-z ]','',x))
review_train['review'][0]

'With all this stuff going down at the moment with MJ ive started listening to his music watching the odd documentary here and there watched The Wiz and watched Moonwalker again Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent Moonwalker is part biography part feature film which i remember going to see at the cinema when it was originally released Some of it has subtle messages about MJs feeling towards the press and also the obvious message of drugs are bad mkaybr br Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of himbr br The actual feature film bit when it finally starts is only on for  minu

In [10]:
#(5) test 데이터셋의 review 변수 전처리하기. 
review_test['review'] = review_test['review'].apply(lambda x: re.sub('[^A-Za-z ]','',x))
review_test['review'][0]

'Naturally in a film whos main themes are of mortality nostalgia and loss of innocence it is perhaps not surprising that it is rated more highly by older viewers than younger ones However there is a craftsmanship and completeness to the film which anyone can enjoy The pace is steady and constant the characters full and engaging the relationships and interactions natural showing that you do not need floods of tears to show emotion screams to show fear shouting to show dispute or violence to show anger Naturally Joyces short story lends the film a ready made structure as perfect as a polished diamond but the small changes Huston makes such as the inclusion of the poem fit in neatly It is truly a masterpiece of tact subtlety and overwhelming beauty'

In [11]:
#12. 데이터셋 분할하기. 
X_train = review_train['review']
X_test  = review_test['review']
y_train = review_train['sentiment'] 

In [12]:
#13. Count Vectorizer로 텍스트 데이터 벡터화 하기. 
#(1) Count Vectorizer 객체 생성하기. 
count_vect = CountVectorizer(stop_words='english', ngram_range=(1,2))

#(2) X_train 데이터셋을 넣어 학습하기. 
count_vect.fit(X_train)

#(3) transform() 메서드를 이용해 벡터화하기. 
X_train_cv = count_vect.transform(X_train)
X_test_cv = count_vect.transform(X_test)

In [18]:
#14. Logistic Regression 모델로 분류하기. + gridsearchCV
#(1) 모델 설정하기. 
lr = LogisticRegression()

#(2) 하이퍼 파라미터 범위 지정하기. 
params = {'C':[1,2,3,4,5]}

#(3) gridsearchCV 모델 설정하기.  
gscv_lr = GridSearchCV(lr, param_grid=params, cv=5)

#(4) 모델 학습하기. 
gscv_lr.fit(X_train_cv,y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [1, 2, 3, 4, 5]})

In [19]:
#(5) 모델의 최적 파라미터 확인하기. 
gscv_lr.best_params_

{'C': 2}

In [20]:
#(6) 모델의 학습 점수 확인하기. 
gscv_lr.best_score_

0.8856000000000002